# Content beased filtering using BERT
Recommendations using BERT

In [10]:
import pandas as pd
import numpy as np

import re
import string

from nltk.classify.textcat import TextCat

import dask.dataframe as dd
import multiprocessing

from keybert import KeyBERT

from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import altair as alt
alt.renderers.enable('mimetype')

from sklearn.metrics.pairwise import cosine_similarity

### Data Cleaning

In [ ]:
path='./Data/'

df_books1=pd.read_csv(path +'Books-old.csv') 
df_books2 = pd.read_csv(path + 'books_1.Best_Books_Ever') #To skip bad lines
df_ratings=pd.read_csv(path + 'Ratings.csv') 
df_users=pd.read_csv(path + 'Users.csv') 

#Renameing
df_books2.rename(columns={'isbn10':'ISBN'}, inplace=True)


C:\Users\Lars\AppData\Local\Temp\ipykernel_23068\3505595599.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books1=pd.read_csv(path +'Books-old.csv')


In [30]:
print("df_books1:")
print(f"Shape: {df_books1.shape}")
print(f"Columns: {df_books1.columns.tolist()}\n")

print("df_books2:")
print(f"Shape: {df_books2.shape}")
print(f"Columns: {df_books2.columns.tolist()}")

df_books1:
Shape: (271360, 8)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']

df_books2:
Shape: (6810, 12)
Columns: ['isbn13', 'ISBN', 'title', 'subtitle', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count']


In [38]:
# Merged dataset



merged_book_df=pd.merge(df_books1, df_books2, on='ISBN', how='inner') 

print("df_books_merged:")
print(f"Shape: {merged_book_df.shape}")
print(f"Columns: {merged_book_df.columns.tolist()}")



merged_book_df.dropna(subset=["description"], inplace=True)

print("df_books_merged:")
print(f"Shape: {merged_book_df.shape}")
print(f"Columns: {merged_book_df.columns.tolist()}")

merged_book_df.head


df_books_merged:
Shape: (2447, 19)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'isbn13', 'title', 'subtitle', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count']
df_books_merged:
Shape: (2369, 19)
Columns: ['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'isbn13', 'title', 'subtitle', 'authors', 'categories', 'thumbnail', 'description', 'published_year', 'average_rating', 'num_pages', 'ratings_count']


<bound method NDFrame.head of             ISBN                                         Book-Title  \
0     0440234743                                      The Testament   
1     0553582909                                           Icebound   
2     0842342702  Left Behind: A Novel of the Earth's Last Days ...   
3     0316769487                             The Catcher in the Rye   
4     067976397X                       Corelli's Mandolin : A Novel   
...          ...                                                ...   
2441  0312155328  From Bondage: Mercy of a Rude Stream (Mercy of...   
2442  0312421974                   This Side of Brightness: A Novel   
2444  0751506451            The Maiden (The Morland Dynasty Series)   
2445  0312890389                   Pacific Edge (Three Californias)   
2446  0439436540             The Time Machine (Scholastic Classics)   

                Book-Author Year-Of-Publication                 Publisher  \
0              John Grisham             

## Data cleanup

In [ ]:
merged_book_df['description'] = merged_book_df['description'].str.replace(r'@\w+', '<user>', regex=True)

merged_book_df['description'] = merged_book_df['description'].str.replace(r'http\w+', '<url>', regex=True)

#list(merged_book_df.description[merged_book_df.Id == 1099555]) #Description with url and html tag

merged_book_df['description'] = merged_book_df['description'].str.replace(r'[{}]+'.format(string.punctuation), '', regex=True)


print(merged_book_df['description'].head())


0    A suicidal billionaire a burntout Washington l...
1    A secret Arctic experiment turns into a frozen...
2    The first book in the authors successful last ...
3    The brilliant funny meaningful novel The New Y...
4    The idyllic world of the Greek island of Cepha...
Name: description, dtype: object
0    A suicidal billionaire a burntout Washington l...
1    A secret Arctic experiment turns into a frozen...
2    The first book in the authors successful last ...
3    The brilliant funny meaningful novel The New Y...
4    The idyllic world of the Greek island of Cepha...
Name: description, dtype: object


In [49]:
from tqdm import tqdm


model=KeyBERT()

def get_keywords_from_description(description):
    keywords = model.extract_keywords(description, keyphrase_ngram_range=(1, 1), stop_words="english")
    keywords = " ".join([k[0] for k in keywords])
    return keywords

get_keywords_from_description(merged_book_df['description'].iloc[20])

'terror fear climax chilling experiences'